In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
#import noisy images and masked images

In [3]:
#conv, concat and outputting latent vector 
def encode(z,z_bw):
    filters = {1:20,2:20,3:60, 4:60, 5:60, 6:60}
    conv1 = tf.layers.conv2d(z, filters[1], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv2 = tf.layers.conv2d(conv1, filters[2], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv1_bw = tf.layers.conv2d(z_bw, filters[1], (3,3),strides=(2,2),padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv2_bw = tf.layers.conv2d(conv1_bw, filters[2], (3,3),strides=(2,2),padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())

    conv = tf.concat([conv2, conv2_bw], 3)
    conv3 = tf.layers.conv2d(conv, filters[3], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv4 = tf.layers.conv2d(conv3, filters[4], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
     
    latent_variable = conv

    return latent_variable

In [6]:
#function for deconv
def decode(x):
    act = tf.nn.relu(x)
    upsample1 = tf.image.resize_nearest_neighbor(act, (512,512))
    conv5 = tf.layers.conv2d(upsample1, 40, (7,7), padding='same',activation=tf.nn.relu)
    upsample2 = tf.image.resize_nearest_neighbor(conv5, (636,636))
    conv6 = tf.layers.conv2d(upsample2, 20, (5,5), padding='same',activation=tf.nn.relu)
    upsample3 = tf.image.resize_nearest_neighbor(conv6, (776,776))
    conv7 = tf.layers.conv2d(upsample3, 3, (7,7), padding='same',activation=tf.nn.relu)
    upsample4 = tf.image.resize_nearest_neighbor(conv7, (1024,1024))
    output = tf.layers.conv2d(upsample4, 3, (7,7), padding='same',activation=tf.nn.relu)

    return output

In [11]:
#import ground truth/ noise free images and their latent distributions

In [12]:
#giving reconst images and ground truth images to discriminator

In [ ]:
def gram_matrix(input_tensor):
    # if input tensor is a 3D array of size Nh x Nw X Nc
    # we reshape it to a 2D array of Nc x (Nh*Nw)
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]

    # get gram matrix 
    gram = tf.matmul(a, a, transpose_a=True)

    return gram

def get_style_loss(base_style, gram_target):
    height, width, channels = base_style.get_shape().as_list()
    gram_style = gram_matrix(base_style)
    # Original eqn as a constant to divide i.e 1/(4. * (channels ** 2) * (width * height) ** 2)
    return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2 * width * height) #(4.0 * (channels ** 2) * (width * height) ** 2)

In [13]:
#reconstruction loss
def reconstruction_loss(x_target, reconstructed_image):
    lr=0.001
    L2_loss = tf.reduce_mean(tf.square(x_target - reconstructed_image))
    style_loss = get_style_loss(reconstructed_image, x_target)
    reconstruction_loss = L2_loss + style_loss
    reconstruction_optimizer = tf.train.AdamOptimizer(learning_rate=lr,
                                                       beta1=0.9).minimize(reconstruction_loss)
    return reconstruction_loss, reconstruction_optimizer

#KL divergence loss
def KLdivergence_loss(encode_LD, groundtruth_LD):
    k = k = tf.keras.losses.KLDivergence()
    kl_loss = k(encode_LD, groundtruth_LD)
    return kl_loss

#wgan loss
def wgan_loss(d_real, d_fake):
    lr=0.01
    disc_loss = tf.reduce_mean(d_real)-tf.reduce_mean(d_fake)
    discriminator_optimizer = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(-disc_loss)
    return disc_loss, discriminator_optimizer

In [ ]:
#outputing the reconstructed image
encode_out = encode(X, X_bw)
reconstructed_image = decode(encode_out)

In [ ]:
reconst_loss, reconst_opt = reconstruction_loss(X_target, reconstructed_image)
kl_loss = KLdivergence_loss(encode_out, gt_LD)
disc_loss, disc_opt = wgan_loss(X_target, reconstructed_image)